<a href="https://colab.research.google.com/github/Volks44/VITMMA19/blob/main/taskdemo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html # was not working !pip install

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch
print(torch.cuda.is_available()) #I dont have cuda, lets use CPU

False


In [ ]:
!wget https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv  #it worked  #IsADirectoryError: [Errno 21] Is a directory: '/content'

--2023-10-31 09:34:44--  https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54288 (53K) [text/plain]
Saving to: ‘insurance.csv’

insurance.csv       100%[===================>]  53.02K  --.-KB/s    in 0.002s  

2023-10-31 09:34:44 (27.1 MB/s) - ‘insurance.csv’ saved [54288/54288]



In [ ]:

!pip install pytorch-lightning #download Pytorch Lightning ( for as an user friendly experience with PyTorch.)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 40.1 MB/s eta 0:00:00


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, r2_score

# Load the dataset from a local path
url = "/content/insurance.csv"
df = pd.read_csv(url)

# Apply one-hot encoding to categorical columns
df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

# Preprocess the dataset
X = df.drop(columns=['charges'])
y = df['charges']
scaler = StandardScaler()
X = scaler.fit_transform(X)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

dataset = TensorDataset(X, y)

# Split the dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)

# Define a simple FeedForward Neural Network using PyTorch Lightning
class FFNN(pl.LightningModule):
    def __init__(self, input_size):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.l1_loss(y_hat, y)  # MAE loss
        self.log('train_loss', loss)
        return loss

# Create the Lightning Trainer with ModelCheckpoint callback
trainer = pl.Trainer(
    max_epochs=10,  # Number of training epochs
    callbacks=[pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', filename='best_model')],
)

# Create the FFNN model and set it to run on CPU
model = FFNN(input_size=X.shape[1])
model = model.to('cpu')

# Start training on CPU
trainer.fit(model, train_dataloader, val_dataloader)

# Load the best model
best_model = FFNN.load_from_checkpoint('best_model.ckpt')

# Make predictions with the best model on the validation set
best_model.eval()
y_pred = best_model(val_dataset.tensors[0])
y_true = val_dataset.tensors[1]

# Calculate MAE and R2 score
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Save the results to a CSV file
results = pd.DataFrame({'MAE': [mae], 'R2': [r2]})
results.to_csv('mlp_results.csv', index=False)

print("MAE:", mae)
print("R2:", r2)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 576   
1 | relu | ReLU   | 0     
2 | fc2  | Linear | 65    
--------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers

Training: |          | 0/? [00:00<?, ?it/s]

<ipython-input-30-b43af805a2ce>:55: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.l1_loss(y_hat, y)  # MAE loss
<ipython-input-30-b43af805a2ce>:55: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.l1_loss(y_hat, y)  # MAE loss
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


FileNotFoundError: ignored

# New Section

In [27]:
# Load the best model
best_model = FFNN.load_from_checkpoint(best_model_checkpoint.best_model_path)

# Make predictions with the best model on the validation set
best_model.eval()
y_pred = best_model.forward(val_dataset.tensors[0])
y_true = val_dataset.tensors[1]
#'/path/to/your/best_model.ckpt'
# Calculate MAE and R2 score
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Specify the full path where you want to save the CSV file
csv_file_path = '/content/mlp_results.csv'  # Replace with your desired directory

# Save the results to the CSV file
results = pd.DataFrame({'MAE': [mae], 'R2': [r2]})
results.to_csv(csv_file_path, index=False)

print("MAE:", mae)
print("R2:", r2)

FileNotFoundError: ignored

In [ ]:
'''import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, r2_score


# Train the model and store the best model
best_model_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    filename='best_model',
    save_last=False  # Don't save the last checkpoint
)

# Create the Lightning Trainer with ModelCheckpoint callback
trainer = pl.Trainer(
    max_epochs=10,  # Number of training epochs
    callbacks=[best_model_checkpoint],
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloader)

# Load the best model
best_model = FFNN.load_from_checkpoint(best_model_checkpoint.best_model_path)

# Make predictions with the best model on the validation set
best_model.eval()
y_pred = best_model.forward(val_dataset.tensors[0])
y_true = val_dataset.tensors[1]

# Calculate MAE and R2 score
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Save the results to a CSV file
results = pd.DataFrame({'MAE': [mae], 'R2': [r2]})
results.to_csv('mlp_results.csv', index=False)

print("MAE:", mae)
print("R2:", r2)
'''

'import torch\nimport torch.nn as nn\nimport torch.optim as optim\nimport pandas as pd\nimport pytorch_lightning as pl\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.preprocessing import OneHotEncoder\nfrom torch.utils.data import DataLoader, TensorDataset\nfrom sklearn.metrics import mean_absolute_error, r2_score\n\n\n# Train the model and store the best model\nbest_model_checkpoint = pl.callbacks.ModelCheckpoint(\n    monitor=\'val_loss\',\n    mode=\'min\',\n    filename=\'best_model\',\n    save_last=False  # Don\'t save the last checkpoint\n)\n\n# Create the Lightning Trainer with ModelCheckpoint callback\ntrainer = pl.Trainer(\n    max_epochs=10,  # Number of training epochs\n    callbacks=[best_model_checkpoint],\n)\n\n# Train the model\ntrainer.fit(model, train_dataloader, val_dataloader)\n\n# Load the best model\nbest_model = FFNN.load_from_checkpoint(best_model_checkpoint.best_model_path)\n\n# Make predict